In [1]:
# show figures inline in the notebook
%matplotlib inline               
import matplotlib.pyplot as plt  # Import library for direct plotting functions
import numpy as np               # Import Numerical Python
from IPython.core.display import display, HTML #Import HTML for formatting output

# NOTE: Uncomment following lines ONLY if you are not using installation via pip
# import sys, os
# rootDir = '/path/to/arc/directory' # e.g. '/Users/Username/Desktop/ARC-Alkali-Rydberg-Calculator'
# sys.path.insert(0,rootDir)

from arc import *                 #Import ARC (Alkali Rydberg Calculator)

In [2]:
#Electron state generator for use with the ARC python package
#Given a Z, this class generates the list of occupied electron states for in input to the ARC energy level calculator

class State:
    #Definition of the principal QN, angular QN, magnetic QN, and spin QN
    def __init__(self,n,l,m,s):
        self.n = n
        self.l = l
        self.m = m
        self.s = s

#Electron state generating class
class ElectronStates:

    #Determines occupancy and returns states given an angular momentum state and unallocated electrons
    def fillStates(self,l):

        #Declares array of possible angular momentum states based on l=n-1 
        self.l = np.linspace(0,l,num=l+1)
        
        #Creates array of magnetic momentum states
        if self.l[-1] == 0:
            self.m = [0]
        else:
            self.m = np.linspace(-(self.l[-1])-.5,self.l[-1]-.5,num= 2*(l+1)-1)
        
        mlen=len(self.m)
        
        #Declares spin states
        self.s = [[0,0] for i in range(mlen)]    
        
        #Iterate through each electron couplet for each magnetic quantum state
        for i in range(mlen*2):

            #Halts if electrons are all used
            if self.ec == 0:
                return False
            
            self.s[int(i%mlen)][int(i/mlen)] = .5*(1 - 2 *int(i/mlen))
            self.ec -= 1

        return True

    
    def aufbau(self,n,l,e_list):
            
            self.fillStates(l)
            for state,couplet in zip(self.m,self.s):
                for spin in couplet:
                    if spin != 0:
                        e_list.append(State(n,l,int(state),spin))
            
            if self.ec == 0:
                return False
            if l != 0:
                return self.aufbau(n+1,l-1,e_list)
            
            else:
                return True
                
    #Class constructor for electron state generating class
    def __init__(self,Z,e_list):

        self.occ_n = []
        self.ec = Z
        n=1
        l=0
        
        while self.ec > 0:
    
            if not self.aufbau(n,l,e_list):
                break
            
            if (n+l)%2 == 0 and n < 5:
                l+=1
            else:
                n+=1



In [3]:
#Declare atom variables
cs=Caesium() #Cesium has 55 electrons w/ valence 7S 
rb=Rubidium() #Rubidium has 37 electrons w/ valences 6S
cs_gnd=55
rb_gnd=37
#States are 1 indexed, not zero indexed
st = []
ElectronStates(200,st)
# for e in st:
#     print(e.n,e.l,e.s)

In [58]:
atom = rb
atom_gnd = rb_gnd
name = "Caesium"

trans_dict={}
pqn = {}
def printTransitionInfo(n1,l1,j1,n2,l2,j2,wvln):
    print("______\nStates")
    printState(n1,l1,j1)
    printState(n2,l2,j2)
    print("\nEnergies")
    print("Initial:    " +str(atom.getEnergy(n1,l1,j1)))
    print("Final:      " + str(atom.getEnergy(n2,l2,j2)))
    print("\n Misc.")
    print("Wavelength: " + str(wvln) + "nm")
    print("Lifetime:   " + str(atom.getStateLifetime(n2,l2,j2)) + "s")
    print("Transition Rate: " +str(atom.getTransitionRate(n1,l1,j1,n2,l2,j2)) +"s^-1")
    
    print("\n")
    
def transitions(wavelen):
    print("Transitions for "+ name + "\n")
    for s1 in st:
        for s2 in st[atom_gnd:]:
            j1 = np.absolute(s1.l+s1.s)
            j2 = np.absolute(s2.l+s2.s)
            if not (s1.n == s2.n and j1 == j2) and not s1.n > s2.n and not (s1.n == s2.n and s1.l >= s2.l):
                wvln = 1000000000*atom.getTransitionWavelength(s1.n,s1.l,j1,s2.n,s2.l,j2)
                if wvln > 200 and wvln < 750:
                    transition_key = str(s1.n)+str(s1.l)+str(j1)+str(s2.n)+str(s2.l)+str(j2)
                    if not transition_key in trans_dict:
                        trans_dict[transition_key]=0 
                        printTransitionInfo(s1.n,s1.l,j1,s2.n,s2.l,j2,wvln)

In [59]:
wavelengths = [500,550]
waist = 1 #The equation is λ/nπsinθ -- so i need a way of determining θ
for wavlen in wavelengths: 
    trans = transitions(wavlen)


Transitions for Caesium

______
States
1   S  1/2
4   F  7/2

Energies
Initial:    -2.8881167545746402
Final:      -0.8553305897389625

 Misc.
Wavelength: 609.9224775392425nm
Lifetime:   3.2838102594204983e-06s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
1   S  1/2
4   F  5/2

Energies
Initial:    -2.8881167545746402
Final:      -0.8553273897389628

 Misc.
Wavelength: 609.9215174043848nm
Lifetime:   6.975723014514004e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
1   S  1/2
5   D  5/2

Energies
Initial:    -2.8881167545746402
Final:      -0.9902989897389625

 Misc.
Wavelength: 653.2987502471941nm
Lifetime:   2.539991092959031e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
1   S  1/2
5   D  3/2

Energies
Initial:    -2.8881167545746402
Final:      -0.9906662897389626

 Misc.
Wavelength: 653.4252129061215nm
Lifetime:   2.7326569783368055e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
St

Transition Rate: 0.0s^-1


______
States
2   P  1/2
4   D  3/2

Energies
Initial:    -7.679157664242629
Final:      -1.7773319897389626

 Misc.
Wavelength: 210.07770177290783nm
Lifetime:   8.709660620720247e-08s
Transition Rate: 0.0s^-1


______
States
2   P  1/2
5   P  3/2

Energies
Initial:    -7.679157664242629
Final:      -2.5880774897389625

 Misc.
Wavelength: 243.53220367128574nm
Lifetime:   2.6434500628062414e-08s
Transition Rate: 0.0s^-1


______
States
2   P  1/2
5   P  1/2

Energies
Initial:    -7.679157664242629
Final:      -2.6175354897389624

 Misc.
Wavelength: 244.94953025324307nm
Lifetime:   2.7835530062774655e-08s
Transition Rate: 0.0s^-1


______
States
2   P  1/2
6   S  1/2

Energies
Initial:    -7.679157664242629
Final:      -1.6810134897389628

 Misc.
Wavelength: 206.70426350107962nm
Lifetime:   4.597625785924859e-08s
Transition Rate: 0.0s^-1


______
States
3   P  3/2
4   D  5/2

Energies
Initial:    -3.617952104442519
Final:      -1.7773872897389627

 Misc.
Wavele

Lifetime:   8.832009561519479e-08s
Transition Rate: 0.0s^-1


______
States
3   P  1/2
5   F  7/2

Energies
Initial:    -3.115591064264404
Final:      -0.5471460897389626

 Misc.
Wavelength: 482.7208627247898nm
Lifetime:   6.722458790231538e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
3   P  1/2
5   F  5/2

Energies
Initial:    -3.115591064264404
Final:      -0.5471436897389625

 Misc.
Wavelength: 482.72041166237676nm
Lifetime:   4.963066969693523e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
3   P  1/2
6   D  5/2

Energies
Initial:    -3.115591064264404
Final:      -0.6200955897389626

 Misc.
Wavelength: 496.8319865215736nm
Lifetime:   2.451494057338682e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
3   P  1/2
6   D  3/2

Energies
Initial:    -3.115591064264404
Final:      -0.6203761897389626

 Misc.
Wavelength: 496.88785788433324nm
Lifetime:   2.5736901431958744e-07s
Transition Rate: 0.0s^-1


____

Lifetime:   3.981378391773e-07s
Transition Rate: 0.0s^-1


______
States
3   D  5/2
8   P  3/2

Energies
Initial:    -3.8844615283829302
Final:      -0.475692893175312

 Misc.
Wavelength: 363.7213629456422nm
Lifetime:   5.728498646528603e-07s
Transition Rate: 0.0s^-1


______
States
3   D  5/2
8   P  1/2

Energies
Initial:    -3.8844615283829302
Final:      -0.4780295723008658

 Misc.
Wavelength: 363.97086157860207nm
Lifetime:   6.418626217259641e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
3   D  5/2
9   S  1/2

Energies
Initial:    -3.8844615283829302
Final:      -0.39571567978587324

 Misc.
Wavelength: 355.3832889439781nm
Lifetime:   2.6804613944340363e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
3   D  5/2
7   F  7/2

Energies
Initial:    -3.8844615283829302
Final:      -0.27885048973896254

 Misc.
Wavelength: 343.8645934560827nm
Lifetime:   9.222043168644151e-07s
Transition Rate: 0.0s^-1


______
States
3   D  5/2
7   F  

Lifetime:   1.216975757435036e-06s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
3   D  3/2
8   F  5/2

Energies
Initial:    -3.878999129898249
Final:      -0.21339694965749195

 Misc.
Wavelength: 338.23691524611627nm
Lifetime:   1.0659489197362746e-06s
Transition Rate: 0.0s^-1


______
States
3   D  3/2
9   D  5/2

Energies
Initial:    -3.878999129898249
Final:      -0.23165402529624396

 Misc.
Wavelength: 339.9299869923777nm
Lifetime:   6.806880317625633e-07s
Transition Rate: 0.0s^-1


______
States
4   P  3/2
5   S  1/2

Energies
Initial:    -9.472965350952137
Final:      -4.177126489738963

 Misc.
Wavelength: 234.1162573968589nm
Lifetime:   infs
Transition Rate: 0.0s^-1


______
States
4   P  1/2
5   S  1/2

Energies
Initial:    -9.691744940412214
Final:      -4.177126489738963

 Misc.
Wavelength: 224.8282424349242nm
Lifetime:   infs
Transition Rate: 0.0s^-1


______
States
5   S  1/2
6   S  1/2

Energies
Initial:    -4.177126489738963
Final:      -1.6810134

Lifetime:   6.418626217259641e-07s
Transition Rate: 0.0s^-1


______
States
5   P  3/2
9   S  1/2

Energies
Initial:    -2.5880774897389625
Final:      -0.39571567978587324

 Misc.
Wavelength: 565.527992841018nm
Lifetime:   2.6804613944340363e-07s
Transition Rate: 0.0s^-1


______
States
5   P  3/2
7   F  7/2

Energies
Initial:    -2.5880774897389625
Final:      -0.27885048973896254

 Misc.
Wavelength: 536.9077938046246nm
Lifetime:   9.222043168644151e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
5   P  3/2
7   F  5/2

Energies
Initial:    -2.5880774897389625
Final:      -0.27884898973896277

 Misc.
Wavelength: 536.9074450467209nm
Lifetime:   7.926929281275776e-07s
6j-Symbol is not triangular!
Transition Rate: 0.0s^-1


______
States
5   P  3/2
8   D  5/2

Energies
Initial:    -2.5880774897389625
Final:      -0.3060957554722984

 Misc.
Wavelength: 543.318097312688nm
Lifetime:   5.011211146654351e-07s
Transition Rate: 0.0s^-1


______
States
5   P  3/2
8   D